In [ ]:
! pip install python-dotenv
! pip install transformers torch huggingface_hub > /dev/null
! pip install torch torchvision > /dev/null
! pip install transformers datasets 
! pip install accelerate>=0.26.0

In [8]:
import pandas as pd 

df=pd.read_csv("fol_dataset_reversed.csv")
column_titles=['english_description','expression']
df=df.reindex(columns=column_titles)
df.to_csv('fol_dataset.csv', index = False)

In [9]:
import os 
from dotenv import load_dotenv

TOKEN = os.getenv("TOKEN")
TOKENIZERS_PARALLELISM = os.getenv("TOKENIZERS_PARALLELISM")
from huggingface_hub import login
login(token = TOKEN)

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it")

In [11]:
from datasets import load_dataset 
dataset = load_dataset('csv', data_files = "fol_dataset.csv")['train']
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_set = split_dataset['train']
test_set = split_dataset['test']

Generating train split: 20000 examples [00:00, 365971.16 examples/s]


In [12]:
print(dataset.column_names)

['english_description', 'expression']


In [13]:
def preprocess(batch):
     inputs = tokenizer(
         batch['expression'],
         max_length=128,
         truncation=True,
         padding = 'max_length',
         
     )    
    
     labels = tokenizer(
         batch['english_description'],
         max_length =128,
         truncation = True, 
         padding = 'max_length'
     )
     inputs['labels']=labels['input_ids']
     return inputs

In [14]:
train_tokenized = train_set.map(preprocess, batched=True)
test_tokenized = test_set.map(preprocess, batched=True)

Map: 100%|██████████| 4000/4000 [00:00<00:00, 10686.23 examples/s]


In [17]:
#setting the trainer 
from transformers import Trainer, TrainingArguments 


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=3,
    learning_rate=2e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)

trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 10.00 GiB. GPU 0 has a total capacity of 47.41 GiB of which 6.72 GiB is free. Process 3479287 has 376.00 MiB memory in use. Process 3479435 has 374.00 MiB memory in use. Process 3481239 has 376.00 MiB memory in use. Process 3481846 has 376.00 MiB memory in use. Including non-PyTorch memory, this process has 39.19 GiB memory in use. Of the allocated memory 37.51 GiB is allocated by PyTorch, and 1.27 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)